In [104]:
import numpy as np
import pandas as pd

# cities = pd.read_csv("canadacities.csv")
greenery = pd.read_csv("../all_subsetted_group.csv")

geo = pd.read_csv("../raw/geodghr4_associations.csv")

GEO = "GEODGHR4"
NDVI = "NDVI"
GEO_NDVI_LABEL = f"{GEO}_{NDVI}_LABEL"
GEO_NDVI_LABELS = f"{GEO_NDVI_LABEL}S"

# LONG = "LONGITUDE"
# LAT = "LATITUDE"

In [105]:
# geo[GEO_NDVI_LABEL] = np.nan
# geo = geo.dropna(subset=[GEO_NDVI_LABELS])

# geo_copy = geo.copy()
# for i, row in geo.iterrows():
#     geo_ndvi_labels = row[GEO_NDVI_LABELS]
#     first_label = geo_ndvi_labels.split(";")[0]

#     geo_copy.loc[i, GEO_NDVI_LABEL] = first_label

# geo = geo_copy.dropna(subset=[GEO_NDVI_LABEL])

In [106]:
# cities[GEO_NDVI_LABEL] = cities[["city", "province_name"]].agg(', '.join, axis=1)
# cities = cities.rename(columns={"lat": LAT, "lng": LONG})

In [107]:
greenery = greenery.loc[greenery["GEN_015"] <= 5]
greenery = greenery.loc[greenery["SDC_015"] <= 2]

greenery = greenery.groupby([GEO, "SDC_015"]).mean()
greenery = greenery.reset_index()

In [108]:
merged_geo_ndvi_label = pd.merge(greenery, geo[[GEO, GEO_NDVI_LABEL]], on=GEO)

In [109]:
# merged_long_lat = pd.merge(merged_geo_ndvi_label, cities[[GEO_NDVI_LABEL, LAT, LONG]], on=GEO_NDVI_LABEL)

In [110]:
geo = merged_geo_ndvi_label[GEO]
merged_geo_ndvi_label[GEO] = np.floor(geo / 1000) * 100 + geo % 100

In [111]:
stats = merged_geo_ndvi_label

g = stats["GEN_015"]
stats["GEN_015"] = 1 - (g - g.min()) / (g.max() - g.min())

n = stats["NDVI"]
stats["NDVI"] = (n - n.min()) / (n.max() - n.min())

In [112]:
N_BINS = 4
bins = np.linspace(0, 1, N_BINS + 1)

for i in range(N_BINS):
    lower = bins[i]
    upper = bins[i + 1]

    inclusive = "right"
    if i == 0:
        inclusive = "both"

    stats.loc[stats["NDVI"].between(lower, upper, inclusive), "NDVI_BINS"] = i 
    stats.loc[stats["GEN_015"].between(lower, upper, inclusive), "GEN_015_BINS"] = i

stats["NDVI"] = stats["NDVI_BINS"]
stats["GEN_015"] = stats["GEN_015_BINS"]

In [113]:
COLUMNS = [
    GEO,
    "GEO_PRV",
    "NDVI",
    "SDC_015",
    "GEN_015"
]

subsetted = stats[COLUMNS]
subsetted.to_csv("visual.csv", index=False)